# Use Case Implementation

The exploration of the smart application concepts consists of the following steps:

1) Requirements Engineering and Planning for Machine Learning
2) Initial Data Pipeline Implementation and Model Training
3) Testing of ML Applications
4) Deployment of ML Applications

So far we have completed the initial requirements engineering and some high level planning for our use cases.
The next step consists of creating an initial implementation stub for your use case that focuses on the ML aspect.

To support this the first sub-section of this notebook contains code for loading the dataset into an sqlite database that should allow you to easily explore and partition the underlying data (using sqlite or another DB as intermediary storage solution between CSV and pandas is an increasingly popular approach).




## 1. Data Loading

To make it easy to slice through and explore there is some code below to load the datasets into sqlite

In [ ]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt

### 1.1 Load Jobs Data

#### Create DB From CSV

 The cell below will create an sqlite DB from the CSV file


In [ ]:
jobs = pd.read_csv('./dfdailyadverts.csv')
conn = sql.connect('job_adverts.db')
# this will give an error message if the db already exists
jobs.to_sql('job_adverts', conn)

#### Establish a Connection to Existing DB

In [ ]:
conn = sql.connect('job_adverts.db')

#### Read Data 

Pandas can be used to execute SQL statements and receive the answer in form of a dataframe.
For large datasets there can be a considerable speed up for filtering and sorting operations when we execute the query in sqlite as compared to pandas.



In [ ]:
# Excecute an SQL query and get back a dataframe
pd.read_sql_query("SELECT description, canton FROM job_adverts WHERE description LIKE '%COVID%'", conn)

In [ ]:
# We can navigate and directly output the results using normal dataframe operations
pd.read_sql_query("SELECT description FROM job_adverts WHERE description LIKE '%COVID%'", conn).iloc[890,0]

In [ ]:
# We can also save the result as a dataframe and use the df as input for further operations
corona_mentions_df = pd.read_sql("SELECT * FROM job_adverts WHERE description LIKE '%COVID%'", conn)
corona_mentions_df.shape

### 1.2 Load E-Commerce Data

#### Create DB From CSV

 The cell below will create an sqlite DB from the CSV file


In [ ]:
jobs = pd.read_csv('./stores_data_UTF8.csv')
conn = sql.connect('stores_data.db')
# this will give an error message if the db already exists
jobs.to_sql('stores_data', conn)

#### Establish a Connection to Existing DB

In [ ]:
conn = sql.connect('stores_data.db')
pd.read_sql_query("SELECT * FROM stores_data LIMIT 5", conn)

#### Read Data 

Pandas can be used to execute SQL statements and receive the answer in form of a dataframe.
For large datasets there can be a considerable speed up for filtering and sorting operations when we execute the query in sqlite as compared to pandas.



In [ ]:
# Excecute an SQL query and get back a dataframe
sql_query_string = "SELECT * FROM stores_data WHERE store_collections LIKE '%COVID%'"
pd.read_sql_query(sql_query_string, conn)

In [ ]:
# We can navigate and directly output the results using normal dataframe operations
pd.read_sql_query(sql_query_string, conn).iloc[2457,4]

In [ ]:
# We can also save the result as a dataframe and use the df as input for further operations
corona_mentions_df = pd.read_sql(sql_query_string, conn)
corona_mentions_df.shape

#### Exploding the Lists in the Shop Data

If you want to expand the lists in the shop data; e.g. in order to use them as labels for training; you can use the following code below.

In [ ]:
corona_mentions_df.assign(store_labels=corona_mentions_df.store_labels.str.replace("[\\[\\]']",'', regex = True).str.split(",")).explode("store_labels")

## 2. Pipeline and Model Training

The main focus of this step consists of building an initial pipeline and training a first model for your use case.

If possible you should try to create multiple models per team. 
The main focus is on getting to the point where you have a trained model that can be evaluated and used for 
predictions. These models and the underlying code will be the input for next week's focus on testing where your tasks will be to:
* Come up with a testing strategy
* Implement tests around the model

When training your models make sure to train models with a subset of the overall available data and hold out a portion of the training data (e.g. 1/3). We will use this portion to simulate the effect of model updates.